In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [2]:
room_info = pd.read_csv('data_preprocessed/room_info_221002.csv')

In [3]:
room_info

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,random_location,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground
0,33446567,5987034,월세,2000,130.0,52.69,57.70,서울시,서대문구,창천동,"37.55975011593247,126.93329296545063",1,2022-09-21 17:21:02,1990.01.16,다세대주택,1990,37.559750,126.933293,0
1,33203240,322799,전세,55000,NaN,29.29,39.67,서울시,서대문구,창천동,"37.55860942977413,126.92758585174795",NaN,2022-09-15 14:04:27,2022.07.08,다세대주택,2022,37.558609,126.927586,0
2,33181218,8485448,월세,5000,200.0,39.08,59.00,서울시,서대문구,창천동,"37.55857456629137,126.92824978616616",5,2022-09-14 15:12:46,2022-07-08,오피스텔,2022,37.558575,126.928250,0
3,33459584,5987034,월세,3000,165.0,72.73,79.34,서울시,마포구,연남동,"37.56050038416363,126.91826753775781",2,2022-09-22 16:21:05,1993-10-06,다가구용 단독주택,1993,37.560500,126.918268,0
4,32955508,17675823,전세,35000,NaN,29.75,39.67,서울시,마포구,창전동,"37.55300474509232,126.92640419917751",2,2022-09-01 21:22:07,20190820,제2종근린생활시설,2019,37.553005,126.926404,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26160,33347615,5748503,월세,1000,140.0,50.45,50.45,서울시,마포구,노고산동,"37.555112040570464,126.93372237850967",5,2022-09-14 13:49:36,2022-06-13,"다세대주택,업무시설(오피스텔)",2022,37.555112,126.933722,0
26161,33442975,37495,월세,1000,75.0,30.08,30.08,서울시,마포구,서교동,"37.55418960217351,126.92495843795764",3,2022-09-22 18:18:41,2002.8.24,다가구주택,2002,37.554190,126.924958,0
26162,33452271,325211,월세,1000,90.0,26.45,26.45,서울시,마포구,연남동,"37.56187727651494,126.92655784969199",3,2022-09-22 11:08:51,20150317,단독주택,2015,37.561877,126.926558,0
26163,33346230,6017035,월세,300,30.0,11.57,11.57,서울시,서대문구,연희동,"37.563030855760196,126.92974609481554",3,2022-09-14 12:46:20,20010523,단독주택,2001,37.563031,126.929746,0


In [4]:
charter_info = room_info[room_info['sales_type'] == '전세'].reset_index(drop=True)

In [5]:
monthly_info = room_info[room_info['sales_type'] == '월세'].reset_index(drop=True)

In [6]:
sum(charter_info['보증금액'] > 50000)/len(charter_info)

0.024471550875066292

In [7]:
charter_info = charter_info[charter_info['보증금액']<= 50000].reset_index(drop=True)

In [8]:
X = charter_info[['전용면적_m2', 'approve_year', 'random_lat', 'random_long', 'is_underground']]
y = charter_info['보증금액']

In [9]:
X

,전용면적_m2,approve_year,random_lat,random_long,is_underground
0,29.75,2019,37.553005,126.926404,0
1,29.51,2019,37.552519,126.926437,0
2,46.28,2001,37.559259,126.925423,0
3,52.89,1996,37.561079,126.917975,0
4,29.96,2021,37.560445,126.927959,0
...,...,...,...,...,...
12871,18.18,2006,37.560043,126.932348,0
12872,18.18,2006,37.560071,126.932342,0
12873,19.83,2011,37.560855,126.921802,0
12874,23.14,1995,37.561718,126.919341,0


## Modeling

In [114]:
train_cols = ['전용면적_m2', 'approve_year', 'random_lat', 'random_long', 'is_underground']

In [102]:
X = charter_info[train_cols]
y = charter_info['보증금액']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [104]:
model = lgbm.LGBMRegressor()

In [105]:
model.fit(X_train, y_train)

LGBMRegressor()

In [106]:
pred = model.predict(X_test)

In [107]:
r2_score(y_test, pred)

0.7849554252676602

## 전세 월세 gap
- 1000만원에 5만원

In [117]:
pred2 = model.predict(monthly_info[train_cols])

In [121]:
monthly_info['전세금액_pred'] = pred2

In [128]:
ratio = (monthly_info['전세금액_pred'] - monthly_info['보증금액'])/monthly_info['월세금액']

In [134]:
monthly_info['ratio']  = ratio

In [146]:
monthly_info[monthly_info['보증금액'] < 5000].sort_values('ratio')

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,...,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground,전세금액_pred,ratio
6578,33019263,3442750,월세,3600,1200.0,322.76,577.19,서울시,용산구,이태원동,...,1,2022-09-15 10:22:07,1988.08,단독주택,1988,37.539021,126.994980,0,41130.890040,31.275742
6616,33406800,31405,월세,1000,80.0,19.83,19.83,서울시,동대문구,제기동,...,반지하,2022-09-19 11:44:53,1992-09-05,단독주택,1992,37.586930,127.035495,1,4231.236498,40.390456
4899,33422573,14508345,월세,480,960.0,230.90,233.28,서울시,강남구,논현동,...,1,2022-09-20 11:11:14,19970509,연립주택,1997,37.520125,127.038833,0,43734.508089,45.056779
3452,33214734,312153,월세,2000,70.0,23.14,23.14,서울시,동대문구,휘경동,...,반지하,2022-09-01 08:01:47,1993.01.29,단독주택,1993,37.592185,127.060094,1,6044.056173,57.772231
5126,33383740,13740988,월세,4000,40.0,23.14,23.14,서울시,강서구,방화동,...,반지하,2022-09-16 17:18:46,2020.03.25,단독주택,2020,37.566682,126.807020,1,6329.846178,58.246154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,33413505,6408183,월세,3000,10.0,33.00,33.00,서울시,강북구,번동,...,2,2022-09-19 15:58:24,19850727,단독주택,1985,37.639892,127.033322,0,11493.561770,849.356177
11423,33405449,5498436,월세,3000,15.0,27.34,27.34,서울시,관악구,신림동,...,옥탑방,2022-09-19 14:17:39,19941021,단독주택,1994,37.484295,126.935737,0,16080.263713,872.017581
10817,33349685,3383939,월세,3000,10.0,23.14,23.14,서울시,관악구,봉천동,...,2,2022-09-14 15:13:12,20020828,제2종근린생활시설,2002,37.475973,126.953953,0,11755.325294,875.532529
5643,33482963,645464,월세,2500,10.0,20.00,20.00,서울시,광진구,화양동,...,1,2022-09-24 14:11:26,1988.09.24,"근린생활시설,주택",1988,37.545233,127.077673,0,11667.584927,916.758493


In [139]:
monthly_info.sort_values('ratio')

,item_id,user_no,sales_type,보증금액,월세금액,전용면적_m2,공급면적_m2,local1,local2,local3,...,floor,updated_at,approve_date,residence_type,approve_year,random_lat,random_long,is_underground,전세금액_pred,ratio
513,33478605,43028,월세,33000,10.0,22.17,66.12,서울시,동대문구,장안동,...,6,2022-09-24 10:25:13,2019-02-21,다세대주택,2019,37.563785,127.060487,0,21055.259044,-1194.474096
10865,33409711,18506028,월세,20000,10.0,19.83,19.83,서울시,관악구,봉천동,...,6,2022-09-23 16:44:30,이번달말 완공예정 미등기,단독주택,2009,37.477192,126.961667,0,11637.069260,-836.293074
4672,33474664,12653,월세,20000,5.0,21.78,30.00,서울시,금천구,독산동,...,5,2022-09-23 17:11:38,20190107,공동주택,2019,37.466502,126.893345,0,16151.264662,-769.747068
11864,33441229,15334096,월세,13400,5.0,14.05,18.00,서울시,관악구,신림동,...,NaN,2022-09-21 13:30:23,2013-12-17,도시형생활주택,2013,37.485042,126.927789,0,9810.054816,-717.989037
8698,33396595,771332,월세,50000,48.0,17.45,33.39,서울시,송파구,방이동,...,NaN,2022-09-17 18:07:28,20191127,오피스텔,2019,37.515201,127.108541,0,20904.983699,-606.146173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6493,33437567,1091258,월세,20000,5.0,56.20,66.12,서울시,서대문구,북가좌동,...,2,2022-09-22 17:49:39,20170517,공동주택,2017,37.580848,126.914452,0,30240.901805,2048.180361
692,33267274,16925783,월세,16800,3.0,38.90,40.00,서울시,강서구,화곡동,...,12,2022-09-05 14:34:35,2018.04.03,공동주택,2018,37.529825,126.841726,0,23530.321470,2243.440490
8075,33391114,17235840,월세,20000,3.0,42.32,50.04,서울시,송파구,방이동,...,4,2022-09-23 13:10:22,20020513,다세대,2002,37.514536,127.111774,0,26790.908396,2263.636132
4960,33448363,11949997,월세,12670,5.0,29.10,29.10,서울시,강남구,논현동,...,4,2022-09-21 18:48:59,20111028,다세대주택,2011,37.517716,127.037697,0,25185.196125,2503.039225


In [130]:
ratio.mean()

225.32477538722998

In [123]:
monthly_info[['보증금액', '월세금액', '전세금액_pred']]

,보증금액,월세금액,전세금액_pred
0,2000,130.0,24766.425820
1,5000,200.0,40914.459450
2,3000,165.0,34343.031451
3,200,150.0,13360.559152
4,260,120.0,14081.624576
...,...,...,...
12961,1000,140.0,41613.209500
12962,1000,75.0,22301.216195
12963,1000,90.0,22090.421955
12964,300,30.0,9991.645990


In [119]:
pred2

array([24766.42581979, 40914.45945038, 34343.03145071, ...,
       22090.42195523,  9991.64598979, 12048.66313637])

## Tuning

In [77]:
def k_fold_evaluation(model, X, y, seed=42):
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    cv_scores = []
    for idx, (train_idx, test_idx) in tqdm(enumerate(cv.split(X, y))):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(
          X_train,
          y_train,
        )
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds, squared=False)
        rsquare = r2_score(y_test, preds)
        cv_scores.append([rmse, rsquare])
    return cv_scores

In [76]:
import optuna